# Option Simplification 

In [1]:
import matplotlib as mpl
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from datetime import datetime
import plotly.express as px
import holidays
import sympy
from scipy import optimize

# Protective Put

- Suppose you own 500 shares of Well Fargo, and buy 5 puts with a strike of 50 for a premium of $\$1.85$. The current stock price is $\$48.92$. 


In [15]:
1800*21.67

39006.0

In [53]:
states = np.linspace(40,60,num = 31)
entry_price = 48.92
shares = 500
#c20 = pd.Series(2000*np.maximum(states-20,0),index= states, name = 'C20')
p50 = pd.Series(500*np.maximum(50-states,0)-925,index = states, name = "P50")
stockpos = pd.Series(shares*states,index = states, name = 'Stock_Pos')
vpnl = pd.Series(shares*entry_price + shares*(states-entry_price),index = states, name= 'Long Stock')

payoffs = pd.concat([p50,vpnl],axis = 1)
payoffs['Portfolio_Value'] = payoffs.sum(axis = 1)
pnl_p=  payoffs['Portfolio_Value'].diff().to_frame("Payoff")
#payoffs['Payoff'] = payoffs['Portfolio_Value'].diff()

fig_protective_put = px.line(payoffs,title="Protective Put")

fig_protective_put.add_vline(x = 50, annotation=dict(text = "Strike = 50"))



In [56]:
px.line(pnl_p,title = 'Profit change Long Stock + Long Put')

- Notice the *green* line and how your losses are protected if the stock price drops below $\$50.00$, however, your breakeven point is actually at $\$48.15$ because you paid $\$1.85$ for the put premium.

- The *green* line is also known as a synthetic **long call with strike equal to 50**. This syntetic long call is priced at $\$0.77$. Why?

    - Because:

    \begin{align}
    C_o = P_o + S_o - K
    \end{align}

In [45]:
c50 = pd.Series((np.maximum(states - 50,0)-0.77)*500,name = 'C50',index = states)
longcallfig = px.line(c50,title=  'Long Call')
longcallfig.add_vline(x = 50,annotation=dict(text = 'Strike = 50'))

# Buy-Write

- A portfolio with a short call option and long stock position
- This is equivalent to a short put option 

<u>Example</u>

$S_0 = \$82.40$ and the number of shares bought is $\Delta = 500$. Additionally, you sold $5C(K=82.5)=\$2.20$

In [74]:
states = np.linspace(78,85,num = 15)
purchase_price = 82.4
delta = 500
c825 = pd.Series(-delta*(np.maximum(states-82.5,0)-2.20),name=  'Short_Call',index= states)
P = pd.Series(500*(states-purchase_price),name= 'Portfolio_Delta',index = states)
TP = pd.concat([P,c825],axis =1)
TP['Payoff'] = TP.sum(axis = 1)
px.line(TP,title ='Buy-Write Total Payoff')



- In order to simulate this exact payoff we need to sell a put for a price of $\$2.30$

- $P_0 = C_0 -S_0 + K$

In [78]:
p825 = pd.Series(-delta*(np.maximum(82.5-states, 0) - 2.3),name = 'Short_Put',index = states)
sputfig= px.line(p825,title = 'Short Put Payoff')
sputfig

- This chart provides the same payoff of selling 5 puts. 

- Thus buying 500 shares of stock and selling 500 calls is equivalent to selling 5 puts with the same strike price. 

# Short Puts Plus Short Stock

Suppose you sell 500 deltas of at $S_0 = \$96.25$ and simultaneoulsy short $5P(K=95) = \$1.45$

- Since this put is all *time value* the price of the corresponding call is equal to the put's time value plus the differential between the strike price and spot price. This equals $(96.25 -95) + 1.45 = \$2.70$.

- Similarly, if you know the price of the ITM call is priced at 2.7, the price of the put with the same strike is equal to the call's **time value** of $\$1.45$.

- This portfolio payoff is exactly equal to $-5C(K = 95) = -\$2.7$

In [92]:
spot = 96.25
state_space = np.linspace(90,100,num = 41)
p95 = -500*(np.maximum(95-state_space,0) - 1.45)
stock_pnl = -500*(state_space-96.25)
P2 = pd.DataFrame([p95,stock_pnl],index = ['Short_Put','Stock_Payoff'],columns=state_space).T
P2['Payoff'] = P2.sum(axis = 1)
fig3 = px.line(P2,title = 'Synthetic Short Call Call')
fig3.add_hline(y = 0, annotation=dict(text ="$0 PNL"))
fig3.add_vline(x = 97.7, annotation=dict(text = "Breakeven at $97.7"))


- Suppose $S_0 = \$21.67$ and you are $\Delta = +1800$.
- Additionally you buy $20C(K = 20)$ and simultaneoulsy short $9P(K= 21)$

- Simplify this portfolio process

In [110]:
spots = np.linspace(10,30,num = 41)
spot = 21.67
delta = 1800
c20 = pd.Series(2000*np.maximum(spots -20,0),index= spots,name ='+C20')
p21 = pd.Series(-900*(np.maximum(21-spots,0)),index = spots,name = '-P21')
stock_delta= pd.Series(delta*(spots-spot),index = spots, name = 'Stock_PNL')
P3 = pd.concat([c20,p21,stock_delta],axis = 1)
P3['PAYOFF'] = P3.sum(axis =1)
fig4 = px.line(P3,title='Porfolio Proces')
fig4


If $S_0= 21.67$ The portfolio has the exact payoff of being long 3800 shares of stock with an average price of $\$20.79$.

\begin{align}

V_{p,t_0} = 3800(S_0 - \$20.79)

\end{align}

- Simplify the portfolio process:
- Stock price is $\$21.67$

\begin{align}

P = 16C(K = 24) + 15P(K = 19) + 1800(S_0 - 21.67)

\end{align}

You are simply long 1800 shares of stock when the stock price is 21.67



In [120]:

c24 = pd.Series(1600*np.maximum(spots -24,0),name = '+C(24)',index = spots)
p19 = pd.Series(1500*np.maximum(19-spots,0),name = "+P(19)",index = spots)
P4 = pd.concat([c24,p19,stock_delta],axis = 1)
P4['Payoff'] = P4.sum(axis=1)

px.line(P4,title = 'Payoff')

- Simplify the portfolio process:
- Stock price is $\$21.67$

\begin{align}

P = -24C(K = 18) - 7P(K = 23) + 1800(S_0 - 21.67)

\end{align}

In [124]:
c18 = pd.Series(-2400*np.maximum(spots -18,0),name = '-C(18)',index = spots)
p23 = pd.Series(-700*np.maximum(23-spots,0),name = "-P(23)",index = spots)
P5 = pd.concat([c18,p23,stock_delta],axis = 1)
P5['Payoff'] = P5.sum(axis=1)
px.line(P5)


- This is equivalent to a short put at strike of k

In [127]:
newspots =np.linspace(0,25,num = 100)

p20= pd.Series(-1300*np.maximum(20.69-newspots,0),index = newspots, name = '-P(20.692)')
px.line(p20)

In [125]:
P5

,-C(18),-P(23),Stock_PNL,Payoff
10.0,-0.0,-9100.0,-21006.0,-30106.0
10.5,-0.0,-8750.0,-20106.0,-28856.0
11.0,-0.0,-8400.0,-19206.0,-27606.0
11.5,-0.0,-8050.0,-18306.0,-26356.0
12.0,-0.0,-7700.0,-17406.0,-25106.0
12.5,-0.0,-7350.0,-16506.0,-23856.0
13.0,-0.0,-7000.0,-15606.0,-22606.0
13.5,-0.0,-6650.0,-14706.0,-21356.0
14.0,-0.0,-6300.0,-13806.0,-20106.0
14.5,-0.0,-5950.0,-12906.0,-18856.0
